In [1]:
import pystan
import numpy as np

In [2]:
ocode = """
data {
    int<lower=1> N;
    real y[N];
}
parameters {
    real mu;
}
model {
    y ~ normal(mu, 1);
}
"""
sm = pystan.StanModel(model_code=ocode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3aaa1aff3be33470f8a5bfa56085d51c NOW.


In [3]:
y2 = np.random.normal(size=20)
np.mean(y2)

0.33886332617228737

In [4]:
# So STAN can deduce mu from the model.
op = sm.optimizing(data=dict(y=y2, N=len(y2)))
op

OrderedDict([('mu', array(0.33886333))])

In [5]:
ocode = '''
parameters {
  real y[2];
}
model {
  y[1] ~ normal(0, 1);
  y[2] ~ double_exponential(0, 2);
}'''

In [7]:
sm = pystan.StanModel(model_code=ocode)
sm

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_42c0ed8065b95a4b695f212f17d17458 NOW.


In [9]:
sm.__dir__()

['model_cppname',
 'model_name',
 'model_code',
 'model_cppcode',
 'model_include_paths',
 'module_name',
 'module',
 'module_filename',
 'module_bytes',
 'fit_class',
 '__module__',
 '__doc__',
 '__init__',
 '__str__',
 'show',
 'dso',
 'get_cppcode',
 'get_cxxflags',
 'get_include_paths',
 '__getstate__',
 '__setstate__',
 'optimizing',
 'sampling',
 'vb',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [11]:
sm.sampling()

Inference for Stan model: anon_model_42c0ed8065b95a4b695f212f17d17458.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
y[1] 5.2e-3    0.02   0.98  -1.97  -0.68   0.01   0.69   1.94   2361    1.0
y[2]   0.05    0.06    2.7  -5.42  -1.36   0.01   1.41   5.85   2292    1.0
lp__  -1.45    0.03   1.15  -4.55  -2.03  -1.18   -0.6   -0.1   1430    1.0

Samples were drawn using NUTS at Mon Oct 14 16:26:25 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).

In [12]:
# bernoulli model
model_code = """
    data {
      int<lower=0> N;
      int<lower=0,upper=1> y[N];
    }
    parameters {
      real<lower=0,upper=1> theta;
    }
    model {
      theta ~ beta(0.5, 0.5);  // Jeffreys' prior
      for (n in 1:N)
        y[n] ~ bernoulli(theta);
    }
"""

In [14]:
from pystan import StanModel

sm = StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_a4c8c0b8415be15f0e6878b909bb73da NOW.


In [15]:
data = dict(N=10, y=[0, 1, 0, 1, 0, 1, 0, 1, 1, 1])

fit = sm.sampling(data=data)
print(fit)

Inference for Stan model: anon_model_a4c8c0b8415be15f0e6878b909bb73da.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.59  3.9e-3   0.15   0.29   0.49    0.6   0.69   0.84   1422    1.0
lp__   -7.99    0.02   0.76 -10.32  -8.18  -7.69   -7.5  -7.44   1939    1.0

Samples were drawn using NUTS at Mon Oct 14 16:30:48 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [18]:
len(fit['theta'])

4000

In [19]:
# reuse model with new data
new_data = dict(N=6, y=[0, 0, 0, 0, 0, 1])
fit2 = sm.sampling(data=new_data)
print(fit2)

Inference for Stan model: anon_model_a4c8c0b8415be15f0e6878b909bb73da.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta   0.21  3.9e-3   0.15   0.02    0.1   0.18    0.3   0.55   1409    1.0
lp__    -4.2    0.02   0.77  -6.39  -4.39  -3.93   -3.7  -3.64   1285    1.0

Samples were drawn using NUTS at Mon Oct 14 16:33:20 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


Older Code follows
Good example modules

In [2]:
fit1 = pystan.stan(model_code=model_code, iter=10)
print(fit1)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_42c0ed8065b95a4b695f212f17d17458 NOW.


Inference for Stan model: anon_model_42c0ed8065b95a4b695f212f17d17458.
4 chains, each with iter=10; warmup=5; thin=1; 
post-warmup draws per chain=5, total post-warmup draws=20.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
y[1]   0.08    0.27   0.76  -1.19  -0.63   0.11   0.62   1.39      8    1.1
y[2]   0.66    0.99   2.34  -3.29  -1.54   1.13   2.08   4.06      6   2.81
lp__  -1.29    0.32   0.67  -2.36  -1.81  -1.12   -0.8  -0.12      4   2.39

Samples were drawn using NUTS at Mon Oct 14 16:03:07 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [3]:
excode = '''
transformed data {
    real y[20];
    y[1] = 0.5796;  y[2]  = 0.2276;   y[3] = -0.2959;
    y[4] = -0.3742; y[5]  = 0.3885;   y[6] = -2.1585;
    y[7] = 0.7111;  y[8]  = 1.4424;   y[9] = 2.5430;
    y[10] = 0.3746; y[11] = 0.4773;   y[12] = 0.1803;
    y[13] = 0.5215; y[14] = -1.6044;  y[15] = -0.6703;
    y[16] = 0.9459; y[17] = -0.382;   y[18] = 0.7619;
    y[19] = 0.1006; y[20] = -1.7461;
}
parameters {
    real mu;
    real<lower=0, upper=10> sigma;
    vector[2] z[3];
    real<lower=0> alpha;
}
model {
    y ~ normal(mu, sigma);
    for (i in 1:3)
    z[i] ~ normal(0, 1);
    alpha ~ exponential(2);
}'''

In [ ]:
def initfun1():
    return dict(mu=1, sigma=4, z=np.random.normal(size=(3, 2)), alpha=1)

exfit0 = pystan.stan(model_code=excode, init=initfun1)

In [ ]:
def initfun2(chain_id=1):
    return dict(mu=1, sigma=4, z=np.random.normal(size=(3, 2)), alpha=1 + chain_id)

exfit1 = pystan.stan(model_code=excode, init=initfun2)

In [ ]:
## The newer way

In [4]:
stanmodelcode = '''
data {
  int<lower=0> N;
  real y[N];
}

parameters {
  real mu;
}

model {
  mu ~ normal(0, 10);
  y ~ normal(mu, 1);
}
'''
r = pystan.stanc(model_code=stanmodelcode, model_name = "normal1")
sorted(r.keys())
r['model_name']

'normal1_86ee00d8bfccf2eb1e1f869b45a42df0'